In [18]:
import datetime as dt
import time
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

import pyxu.abc as pxa
import pyxu.operator as pxop
import pyxu.opt.stop as pxos
#from pyxu.opt.solver.pgd import PGD

import pfw_hawkes as pyfwl
from hawkes_likelihood import HawkesLikelihood
import utils as ut

seed = 1  # None  # for reproducibility
if seed is None:
    seed = np.random.randint(0, 1000)
print(f"Seed: {seed}")

Seed: 1


In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib qt

In [ ]:
testarr = np.array([1, 9, 9, 9, 2, 9, 9, 9, 3, 9, 9, 9]) #M=3 => M*(M+1) = 12
#[kM + k, kM+k + M] pour la k'eme tranche
indices = [index for k in range(3) for index in range(k*3 + k + 1, k*3 + k + 3 + 1)]
print(indices)

print([testarr[(k*3 + k + 1):(k*3 + k + 3 + 1)] for k in range(0, 3)])
print(testarr[indices])

In [24]:
# ne pas oublier que le vecteur de parametres theta est dans R^{M^2 + M}
testL1norm = ut.L1NormPartialReg(shape=(1,3*(3+1)), S=indices, regLambda=1)

In [27]:
print(testL1norm(testarr))
print(testL1norm.prox(testarr, tau=1))

81.0
[1. 8. 8. 8. 2. 8. 8. 8. 3. 8. 8. 8.]


In [3]:
# Parameter for reconstruction
#lambda_factor = 0.1

## Parameters of the solvers
# Base
remove = True
min_iterations = 1
# Stop
eps = 1e-4
tmax = 15.0
eps_dcv = 1e-2
# PFW
ms_threshold = 0.8
init_correction = 1e-1
final_correction = 1e-6
correction_steps = 5

### Generation of the source

In [4]:
path = "simulated_data/events0.csv"
hpL = HawkesLikelihood(path=path)

In [108]:
start = time.time()
lip = hpL.opA.estimate_lipschitz(method='svd')
print("Computation time of the Lipschitz constant: {:.4f}".format(time.time()-start))
print("Lipschitz constant:", lip)
#np.linalg.svd to compute the largest singular value which is = to the lipschitz constant for linear operators

Computation time of the Lipschitz constant: 0.0260
Lipschitz constant: 205.2870358282836


In [107]:
# blockDiagA = sp.linalg.block_diag(hpL.A[0], hpL.A[1], hpL.A[2], hpL.A[3], hpL.A[4], hpL.A[5],
#                                   hpL.A[6], hpL.A[7])
# _, s, _ = np.linalg.svd(blockDiagA)
# print(s)
# lip = np.max(s)
# print(blockDiagA.shape)

In [91]:
#lambda_ = lambda_factor * np.linalg.norm(op.T(measurements), np.infty)  # rule of thumb to define lambda
lambda_ = 10e-5

### Defining solvers

In [92]:
pfw = pyfwl.PFWLasso(
    hpL,  # replace by likelihood operator hpL
    hpL.opA,  # replace by hpL.opA
    lambda_,
    ms_threshold=ms_threshold,
    init_correction_prec=init_correction,
    final_correction_prec=final_correction,
    remove_positions=remove,
    min_correction_steps=correction_steps,
    show_progress=False,
)

In [93]:
stop_crit = pxos.RelError(
    eps=eps,
    var="objective_func",
    f=None,
    norm=2,
    satisfy_all=True,
)
# alternative stopping criteria
dcv = pyfwl.dcvStoppingCrit(eps_dcv)

# Minimum number of iterations
min_iter = pxos.MaxIter(n=min_iterations)

# Maximum duration
max_duration = pxos.MaxDuration(t=dt.timedelta(seconds=tmax))

stop = (min_iter & stop_crit) | max_duration

# track DCV
track_dcv = pxos.AbsError(eps=1e-10, var="dcv", f=None, norm=2, satisfy_all=True)

### Solving

In [94]:
print("Polyatomic FW: Solving ...")
start = time.time()
pfw.fit(stop_crit= stop | track_dcv, diff_lipschitz=lip**2)
data_p, hist_p = pfw.stats()
time_p = time.time() - start
print("\tSolved in {:.3f} seconds".format(time_p))


Polyatomic FW: Solving ...
diff_lipschitz constant provided.
	Solved in 0.055 seconds


### Evaluation of the solve

In [95]:
print("Final value of dual certificate:\n\tPFW: {:.4f}".format(data_p["dcv"]))
print(
    "Final value of objective function:\n\tPFW : {:.2f}".format(hist_p[-1][-1]))

Final value of dual certificate:
	PFW: -316752137604238.8750
Final value of objective function:
	PFW : 37322.60


In [96]:
print(data_p["x"])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
